# CIFAR 10 ConvNet model

### Preprocessing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from keras.utils import to_categorical, plot_model
from sklearn.preprocessing import StandardScaler
import os
import datetime
import time
import tensorboard
%load_ext tensorboard


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train = x_train / 255
#x_test = x_test / 255


y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print("Tensorflow version: ",tf.__version__)
print(x_train.shape)


# Training model with model.fit method

### callbacks

In [ ]:
from tensorflow.keras.callbacks import Callback, EarlyStopping, TensorBoard, ModelCheckpoint

logdir = 'logs/hparms'
chekpoints_dir = os.path.join('chekpoints', "ckpt_{epoch}")
batch_size = 125 # for having the rest to zero (50000 % 125 = 0)


# defining the callbacks
tensorboard_callback = TensorBoard(log_dir=logdir, histogram_freq=1, update_freq='epoch', profile_batch = 100000000)  # log metrics
#Keras_callback =   # log hparams
earlyStopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# Save model after every 3 epochs (150000 / 50000)
checkpoint_callback = ModelCheckpoint(filepath = chekpoints_dir, save_freq=150000, monitor='val_loss',save_best_only=True, verbose = 1)

class TrainCallback(Callback):

    def __init__(self, x_train, y_train):
        super().__init__()
        self.x_train = x_train
        self.y_train = y_train
    
    def on_epoch_begin(self, epoch, logs=None):
        print("<-----------------------------------------------       EPOCH {}     ----------------------------------------------->".format(epoch+1))
    
    def on_epoch_end(self, epoch, logs=None):
        (loss, acc) = self.model.evaluate(self.x_train, self.y_train, batch_size = 8192, verbose=0, callbacks=[checkpoint_callback])
        print(f"Real Loss on train : {loss}")
        print(f"Real Acc on train : {acc}")
    
    def on_test_begin(self, epoch, logs=None):
        print("\nCalculating the real train loss/accuracy ...")
    

### Define the model

In [ ]:
from tensorflow.keras.layers import Input, Add, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, softmax
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, RMSprop

def conv_model(hparams):
    
    dropout_rate = int(hparams[HP_DROPOUT])*0.1
    
    input = Input((32, 32, 3), name='input')
    
    # adding or not batch normalization layer
    if(hparams[HP_BATCH_NORM] == 'yes'):
        batchNorm_1 = BatchNormalization(name='batchNorm_1')(input)
        conv_1 = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu',kernel_regularizer=l2(0.001), name='conv_1')(batchNorm_1)
    else:
        conv_1 = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu',kernel_regularizer=l2(0.001), name='conv_1')(input)
    
    pool_1 = MaxPooling2D(pool_size=(2, 2), padding='same', name='pool_1')(conv_1)
    
    conv_2 = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(0.001), name='conv_2')(pool_1)
    pool_2 = MaxPooling2D(pool_size=(2, 2), padding='same', name='pool_2')(conv_2)
    
    flatten = Flatten(name='flatten')(pool_2)
    fc_1 = Dense(256, activation='linear', name='fc_1')(flatten)
    dropout_1 = Dropout(dropout_rate, name='dropout_1')(fc_1)
    relu_1 = Activation('relu', name='relu_1')(dropout_1)
    fc_2 = Dense(256, activation='relu', name='fc_2')(relu_1)
    dropout_2 = Dropout(dropout_rate, name='dropout_2')(fc_2)
    relu_2 = Activation('relu', name='relu_2')(dropout_2)
    output = Dense(10, activation='softmax', name='output')(relu_2)
    model = Model(input, output)
    
    if(hparams[HP_OPTIMIZER] == 'Adam'):
        model.compile(
          optimizer=Adam(learning_rate=float(hparams[HP_LR])),
          loss='categorical_crossentropy',
          metrics=['accuracy'],
        )
    else:
        model.compile(
          optimizer=RMSprop(learning_rate=float(hparams[HP_LR])),
          loss='categorical_crossentropy',
          metrics=['accuracy'],
        )
    
    hist = model.fit(x_train, y_train , epochs=30,
                     batch_size=batch_size,
                     validation_data=(x_test, y_test),
                     callbacks=[tensorboard_callback, earlyStopping ,TrainCallback(x_train, y_train)])
    
    return hist.history['val_accuracy'][-1] # returning the last value of validation accuracy

### Define our hyperparameters

In [ ]:
from tensorboard.plugins.hparams import api as hp

HP_BATCH_NORM = hp.HParam('batch_norm_active', hp.Discrete(['yes', 'no']))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete(['0', '2', '5']))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['Adam', 'RMSprop']))
HP_LR = hp.HParam('learning_rate', hp.Discrete(['0.01', '0.001']))


METRICS = 'accuracy'

with tf.summary.create_file_writer(logdir).as_default():
    hp.hparams_config(
        hparams=[HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRICS, display_name='Accuracy')],
    )

### Training

In [ ]:
def train(train_dir, hparams):
    with tf.summary.create_file_writer(train_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = conv_model(hparams)
        tf.summary.scalar(METRICS, accuracy, step=1)


session_num = 1

# varying the optimizers 
for optimizer in HP_OPTIMIZER.domain.values:
    start_time = time.time()
    hparams = {
        HP_LR: 0.01,
        HP_BATCH_NORM: 'no',
        HP_DROPOUT: 0,
        HP_OPTIMIZER: optimizer,
    }
    print("<=========================================       RUN {}      =================================================>".format(session_num))
    run_name = "run-%d" % session_num
    print({h.name: hparams[h] for h in hparams})
    train(logdir +"/"+ run_name, hparams)
    print('duration : {}min' .format(int((start_time - time.time()) / 60)))
    session_num += 1  

# varying the batch normalization     
for bn in HP_BATCH_NORM.domain.values:
    start_time = time.time()
    hparams = {
        HP_LR: 0.01,
        HP_BATCH_NORM: bn,
        HP_DROPOUT: 0,
        HP_OPTIMIZER: 'Adam'
    }
    print("<=========================================       RUN {}      =================================================>".format(session_num))
    run_name = "run-%d" % session_num
    print({h.name: hparams[h] for h in hparams})
    train(logdir +"/"+ run_name, hparams)
    print('duration : {}min' .format(int((start_time - time.time()) / 60)))
    session_num += 1                
                
# varying the dropout 
for dropout_rate in HP_DROPOUT.domain.values:
    start_time = time.time()
    hparams = {
        HP_DROPOUT: dropout_rate,
        HP_LR: 0.01,
        HP_BATCH_NORM: 'no',
        HP_OPTIMIZER: 'Adam'
    }
    print("<=========================================       RUN {}      =================================================>".format(session_num))
    run_name = "run-%d" % session_num
    print({h.name: hparams[h] for h in hparams})
    train(logdir +"/"+ run_name, hparams)
    print('duration : {}min' .format(int((start_time - time.time()) / 60)))
    session_num += 1     
    
# varying the learning rate
for lr in HP_LR.domain.values:
    start_time = time.time()
    hparams = {
        HP_LR: lr,
        HP_BATCH_NORM: 'no',
        HP_DROPOUT: 0,
        HP_OPTIMIZER: 'Adam',
    }
    print("<=========================================       RUN {}      =================================================>".format(session_num))
    run_name = "run-%d" % session_num
    print({h.name: hparams[h] for h in hparams})
    train(logdir +"/"+ run_name, hparams)
    print('duration : {}min' .format(int((start_time - time.time()) / 60)))
    session_num += 1        

print("\n\n<=========================================       END      =================================================>")    

In [ ]:
%tensorboard --logdir logs

# FIN FIN FIN FIN FIN FIN FIN FIN FIN FIN FIN FIN FIN FIN 

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

# Training model with gradient tape method

In [ ]:

def normalize(x, y):
    x = tf.image.per_image_standardization(x)
    return x, y


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))


train_dataset = train_dataset.shuffle(buffer_size=50000).map(normalize).batch(batch_size)   
test_dataset = test_dataset.batch(batch_size)
print(train_dataset)

### Creating the model

    add image representation for model from plt.model_plot

### Preprocessing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from keras.utils import to_categorical, plot_model
from sklearn.preprocessing import StandardScaler
import os
import datetime
import time
import tensorboard
%load_ext tensorboard


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train = x_train / 255
#x_test = x_test / 255


#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

def normalize(x, y):
    x = tf.image.per_image_standardization(x)
    return x, y


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))


batch_size = 250
train_dataset = train_dataset.shuffle(buffer_size=50000).map(normalize).batch(batch_size)   
test_dataset = test_dataset.batch(batch_size)
print(train_dataset)

print("Tensorflow version: ",tf.__version__)

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Add, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, softmax

def conv_model():
    
    input = Input((32, 32, 3), name='input')

    conv_1 = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', name='conv_1', input_shape = (32, 32, 3), data_format="channels_last")
    pool_1 = MaxPooling2D(pool_size=(2, 2), padding='same', name='pool_1')(conv_1)
    
    conv_2 = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', name='conv_2')(pool_1)
    pool_2 = MaxPooling2D(pool_size=(2, 2), padding='same', name='pool_2')(conv_2)
    
    conv_3 = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', name='conv_3')(pool_2)
    pool_3 = MaxPooling2D(pool_size=(2, 2), padding='same', name='pool_3')(conv_3)
    
    flatten = Flatten(name='flatten')(pool_3)
    fc_1 = Dense(512, activation='relu', name='fc_1')(flatten)
    fc_2 = Dense(256, activation='relu', name='fc_2')(fc_1)
    fc_3 = Dense(128, activation='relu', name='fc_3')(fc_2)
    output = Dense(10, activation='softmax', name='output')(fc_3)
    
    model = Model(conv_1, output)
 
    return model

model = conv_model()
print(model.summary())

In [ ]:
#plot_model(model, "cifar10_CNN.png")
# set metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
from tensorflow.keras.callbacks import Callback

# set the optimizer
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)


class TrainCallback(Callback):

    def __init__(self, model, x_train, y_train):
        super().__init__()
        self.x_train = x_train
        self.y_train = y_train
        self.model = model

    def on_epoch_end(self, epoch, logs=None):
        (loss, acc) = self.model.evaluate(self.x_train, self.y_train, batch_size = batch_size)
        print(f"Real Loss on train : {loss}")
        print(f"Real Acc on train : {acc}")

#@tf.function
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        
        predictions = model(x_train, training=True)  # like model.Fit in keras
        loss = loss_object(y_train, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    TrainCallback(model, x_train, y_train)

    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [ ]:


current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'cifar10/CNN_' + current_time + '/train/'
test_log_dir = 'cifar10/CNN_' + current_time + '/train/'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
epochs = 3

model = conv_model()

print("Start training ...")
for epoch in range(epochs):
    print("<-----------------------------------------------       EPOCH {}     ----------------------------------------------->".format(epoch+1))
    print("Calculating loss/accuracy ...")
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    # to record data in tensorboard
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    print('Epoch {}, Loss: {:.3f}, Accuracy: {:.3f}, Test Loss: {:.3f}, Test Accuracy: {:.3f}'.format(epoch+1,train_loss.result(), train_accuracy.result()*100,test_loss.result(), test_accuracy.result()*100))
    if((epoch + 1) % 3 == 0):
        try:
            last_model = 'model_ep'+str(epoch)+'.h5'
            model.save(last_model)
            print('Saved model to disk ({})', last_model)
        except:
            print('error occurred when saving model ({})', last_model)

print("<-----------------------------------------------       END     ----------------------------------------------->")
# Reset metrics every epoch
train_loss.reset_states()
test_loss.reset_states()
train_accuracy.reset_states()
test_accuracy.reset_states()



In [ ]:
%tensorboard --logdir 'cifar10/'

In [ ]:
loaded_model = load_model(last_model)

# let's run some tests

In [ ]:
import cv2
import numpy as np
from keras.models import load_model

img_row, img_height, img_depth = 32,32,3
classifier = load_model("C:/Users/Asus/DeepLearningCV/Trained Models/cifar_simple_cnn.h5")
color = True
scale = 8

def draw_test(name, res, input_im, scale, img_row, img_height):
    BLACK = [0,0,0]
    res = int(res)
    if res == 0:
        pred = "airplane"
    if res == 1:
        pred = "automobile"
    if res == 2:
        pred = "bird"
    if res == 3:
        pred = "cat"
    if res == 4:
        pred = "deer"
    if res == 5:
        pred = "dog"
    if res == 6:
        pred = "frog"
    if res == 7:
        pred = "horse"
    if res == 8:
        pred = "ship"
    if res == 9:
        pred = "truck"
        
    expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0]*2 ,cv2.BORDER_CONSTANT,value=BLACK)
    if color == False:
        expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image, str(pred), (300, 80) , cv2.FONT_HERSHEY_COMPLEX_SMALL,4, (0,255,0), 2)
    cv2.imshow(name, expanded_image)


for i in range(0,10):
    rand = np.random.randint(0,len(x_test))
    input_im = x_test[rand]
    imageL = cv2.resize(input_im, None, fx=scale, fy=scale, interpolation = cv2.INTER_CUBIC) 
    input_im = input_im.reshape(1,img_row, img_height, img_depth) 
    
    ## Get Prediction
    res = str(classifier.predict_classes(input_im, 1, verbose = 0)[0])
    
    draw_test("Prediction", res, imageL, scale, img_row, img_height) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [ ]:
batch_size = 64
epochs = 150

@tf.function  # graph mode
def train_(batch_images, targets):
    with tf.GradientTape() as tape:

        predictions = model(batch_images)
        loss = loss_object(targets, predictions) # get loss of these predictions
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # to join each gradient
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(targets, predictions)
    

In [ ]:

# https://www.tensorflow.org/api_docs/python/tf/GradientTape
tf.reset_default_graph()

# Inputs
input = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input')
output =  tf.placeholder(tf.float32, shape=(None, 10), name='output')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')


model.fit(x_train, y_train,
         batch_size = 64,
         epochs = 150,
         shuffle = True,
         validation_data = (x_test,y_test))



In [ ]:
conv1 = Conv2D(64,(3,3), strides=(1,1), padding='same', activation='relu',name='conv1')(input)
    conv1 = Conv2D(64,(3,3), strides=(1,1), padding='same', activation='relu',name='conv1')(input)
    
    
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), activation='relu', input_shape=x_train.shape[1:]))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    